In [1]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from time import sleep, strftime

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

current_dir = '../Daily'
output_dir = os.path.join(os.path.dirname(current_dir), 'Data')
file_out = os.path.join(output_dir, "ports-hilo-df.csv")
print(file_out)

..\Data\ports-hilo-df.csv


In [2]:
sql = """
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name
"""
in_df = pd.read_sql(sql, const)
in_df.shape

(32, 1)

In [3]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  05:51 PM


In [4]:
url = "https://www.set.or.th/en/market/product/stock/quote/"

output_columns = ['name', 'today_low', 'today_high', 'year_low', 'year_high']
# Initialize an empty DataFrame
out_df = pd.DataFrame(columns=output_columns)

for index, row in in_df.iterrows():
    name = str(row['name']).upper()
    response = request.urlopen(url + name + '/price')
    html_page = BeautifulSoup(response, 'html.parser')
        
    values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
    today_low = values_low[0].text
    if (today_low == '-'):
        today_low = '0.00'
        
    values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
    today_high = values_high[0].text
    if (today_high == '-'):
        today_high = '0.00'         
        
    year_low = values_low[1].text        
    year_high = values_high[1].text    
    
    # Create a temporary DataFrame for the current row
    temp_df = pd.DataFrame([[name, today_low, today_high, year_low, year_high]], columns=output_columns)
    # Concatenate the temporary DataFrame with the main DataFrame
    out_df = pd.concat([out_df, temp_df], ignore_index=True)
    
    sleep(2)   # to avoid overloading the server and getting blocked
    print([name, today_low, today_high, year_low, year_high]) 
        
out_df

['AH', '29.75', '30.50', '25.50', '39.25']
['AIMIRT', '10.40', '10.80', '10.00', '12.50']
['ASK', '19.90', '20.10', '19.20', '34.75']
['ASP', '2.80', '2.86', '2.68', '3.24']
['AWC', '3.82', '3.96', '3.40', '6.20']
['BCH', '22.00', '22.40', '16.60', '23.00']
['BEM', '7.40', '7.60', '7.50', '10.10']
['CPNREIT', '11.10', '11.40', '9.40', '21.30']
['CRC', '35.00', '37.25', '36.25', '47.50']
['DIF', '8.10', '8.30', '7.25', '13.70']
['GVREIT', '6.10', '6.25', '5.45', '9.90']
['IVL', '23.60', '24.20', '23.00', '42.50']
['JASIF', '6.20', '6.35', '5.70', '8.30']
['JMART', '16.00', '16.50', '13.10', '41.25']
['JMT', '25.00', '25.75', '23.30', '67.75']
['KCE', '47.75', '49.75', '35.50', '58.50']
['MCS', '6.35', '6.50', '5.90', '10.30']
['NER', '5.05', '5.20', '4.40', '6.70']
['ORI', '8.35', '8.55', '8.05', '12.90']
['PTG', '8.75', '8.95', '7.70', '14.90']
['PTTGC', '34.25', '35.25', '31.75', '52.50']
['RCL', '25.75', '26.50', '18.50', '33.00']
['SCC', '272.00', '275.00', '272.00', '358.00']
['SEN

,name,today_low,today_high,year_low,year_high
0,AH,29.75,30.50,25.50,39.25
1,AIMIRT,10.40,10.80,10.00,12.50
2,ASK,19.90,20.10,19.20,34.75
3,ASP,2.80,2.86,2.68,3.24
4,AWC,3.82,3.96,3.40,6.20
5,BCH,22.00,22.40,16.60,23.00
6,BEM,7.40,7.60,7.50,10.10
7,CPNREIT,11.10,11.40,9.40,21.30
8,CRC,35.00,37.25,36.25,47.50
9,DIF,8.10,8.30,7.25,13.70


In [5]:
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')

In [6]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  05:53 PM


In [7]:
out_df.query('today_low <= year_low')

,name,today_low,today_high,year_low,year_high
6,BEM,7.4,7.60,7.50,10.1
8,CRC,35.0,37.25,36.25,47.5
22,SCC,272.0,275.00,272.00,358.0


In [8]:
out_df.query('today_low <= year_low').shape[0]

3

In [9]:
out_df.query('today_high >= year_high')

,name,today_low,today_high,year_low,year_high


In [10]:
out_df.query('today_high >= year_high').shape[0]

0

In [11]:
out_df.to_csv(file_out, index=False)  